In [ ]:
# Based on https://github.com/weiaicunzai/pytorch-cifar100
# and https://github.com/junyuseu/pytorch-cifar-models/tree/master

In [1]:
import time
import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

# Models

## Baseline Module

In [2]:
class PreActBasic(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride):
        super(PreActBasic, self).__init__()

        self.residual = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels*self.expansion, 3, padding=1, bias=False)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Conv2d(in_channels, out_channels * self.expansion, 1, stride=stride, bias=False)

    def forward(self, x):
        res = self.residual(x)
        mapping = self.shortcut(x)

        return res + mapping

In [3]:
class PreActBottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride):
        super(PreActBottleneck, self).__init__()

        self.residual = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, 1, stride=stride, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * self.expansion, 1, bias=False)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Conv2d(in_channels, out_channels * self.expansion, 1, stride=stride, bias=False)

    def forward(self, x):
        res = self.residual(x)
        mapping = self.shortcut(x)

        return res + mapping

## ReLU Dropped Module

In [4]:
class ReLUDroppedBasic(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride):
        super(ReLUDroppedBasic, self).__init__()

        self.residual = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels*self.expansion, 3, padding=1, bias=False)
        )

        self.reludrop = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels*self.expansion, 3, padding=1, bias=False)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Conv2d(in_channels, out_channels * self.expansion, 1, stride=stride, bias=False)

    def forward(self, x):
        res = self.residual(x)
        reludropres = self.reludrop(x)
        mapping = self.shortcut(x)

        return res + reludropres + mapping

In [5]:
class ReLUDroppedBottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride):
        super(ReLUDroppedBottleneck, self).__init__()

        self.residual = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, 1, stride=stride, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * self.expansion, 1, bias=False)
        )

        self.reludrop = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.Conv2d(in_channels, out_channels, 1, stride=stride, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False),

            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels * self.expansion, 1, bias=False)
        )

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels * self.expansion:
            self.shortcut = nn.Conv2d(in_channels, out_channels * self.expansion, 1, stride=stride, bias=False)

    def forward(self, x):
        res = self.residual(x)
        reludropres = self.reludrop(x)
        mapping = self.shortcut(x)

        return res + reludropres + mapping

## Network Definition

In [6]:
class PreActResNet(nn.Module):
    def __init__(self, block, num_block, num_classes=100):
        super(PreActResNet, self).__init__()
        self.input_channels = 16

        self.pre = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True)
        )

        self.stage1 = self._make_layers(block, num_block[0], 16, 1)
        self.stage2 = self._make_layers(block, num_block[1], 32, 2)
        self.stage3 = self._make_layers(block, num_block[2], 64, 2)

        self.finalact = nn.Sequential(
            nn.BatchNorm2d(64*block.expansion),
            nn.ReLU(inplace=True)
        )

        self.linear = nn.Linear(64*block.expansion, num_classes)

    def _make_layers(self, block, block_num, out_channels, stride):
        strides = [stride] + [1]*(block_num-1)
        layers = []

        for stride in strides:
            layers.append(block(self.input_channels, out_channels, stride))
            self.input_channels = out_channels * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.pre(x)

        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)
        x = self.finalact(x)

        x = F.avg_pool2d(x, 8)
        x = x.view(x.size(0), -1)
        x = self.linear(x)

        return x

In [7]:
def basicresnet110(num_classes=10):
    return PreActResNet(PreActBasic, [18, 18, 18], num_classes)

def bottleneckresnet164(num_classes=10):
    return PreActResNet(PreActBottleneck, [18, 18, 18], num_classes)

def basicresnet56_reludrop(num_classes=10):
    return PreActResNet(ReLUDroppedBasic, [9, 9, 9], num_classes)

def bottleneckresnet83_reludrop(num_classes=10):
    return PreActResNet(ReLUDroppedBottleneck, [9, 9, 9], num_classes)

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(count_parameters(basicresnet110(10)))
print(count_parameters(bottleneckresnet164(10)))
print(count_parameters(basicresnet56_reludrop(10)))
print(count_parameters(bottleneckresnet83_reludrop(10)))

1730554
1703290
1707418
1691802


# Utils

In [ ]:
def get_train_dataloader(mean, std, batch_size=128, num_classes=10, num_workers=4, shuffle=True):
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    if num_classes == 100:
        cifar100_train = torchvision.datasets.CIFAR100(
            root='./data', train=True, download=True, transform=transform_train)
        cifar100_train_loader = DataLoader(
            cifar100_train, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size)
        return cifar100_train_loader

    if num_classes == 10:
        cifar10_train = torchvision.datasets.CIFAR10(
            root='./data', train=True, download=True, transform=transform_train)
        cifar10_train_loader = DataLoader(
            cifar10_train, shuffle=shuffle, num_workers=num_workers, batch_size=batch_size)
        return cifar10_train_loader

def get_test_dataloader(mean, std, batch_size=100, num_classes=10, num_workers=4, shuffle=False):
    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    if num_classes == 100:
        cifar100_test = torchvision.datasets.CIFAR100(
            root='./data', train=False, download=True, transform=transform_test)
        cifar100_test_loader = DataLoader(
            cifar100_test, shuffle=shuffle, num_workers = num_workers, batch_size=100)
        return cifar100_test_loader

    if num_classes == 10:
        cifar10_test = torchvision.datasets.CIFAR10(
            root='./data', train=False, download=True, transform=transform_test)
        cifar10_test_loader = DataLoader(
            cifar10_test, shuffle=shuffle, num_workers = num_workers, batch_size=100)
        return cifar10_test_loader

In [ ]:
# https://github.com/Armour/pytorch-nn-practice/blob/master/utils/meanstd.py

def compute_mean_std(num_classes=10):
    mean = [0.49139968, 0.48215841, 0.44653091]
    std = [0.24703223, 0.24348513, 0.26158784]

    if num_classes == 100:
        mean = [0.50707516, 0.48654887, 0.44091784]
        std = [0.26733429, 0.25643846, 0.27615047]

    return mean, std

In [ ]:
def lr_schedule(epoch):
    if epoch < 1:
        return 0.1
    elif epoch < 80:
        return 1
    elif epoch < 120:
        return 0.1 ** 1
    else:
        return 0.1 ** 2

# Training

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def modelload():
    num_classes = 10

    # net = basicresnet110(num_classes=num_classes).to(device)
    # net = bottleneckresnet164(num_classes=num_classes).to(device)
    net = basicresnet56_reludrop(num_classes=num_classes).to(device)
    # net = bottleneckresnet83_reludrop(num_classes=num_classes).to(device)

    return num_classes, net

In [ ]:
def utilload():
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
    scheduler = LambdaLR(optimizer, lr_lambda = lr_schedule)

    batch_size = 128
    num_epochs = 165

    return criterion, optimizer, scheduler, batch_size, num_epochs

In [ ]:
def dataload():
    mean, std = compute_mean_std(num_classes)
    cifar_train_loader = get_train_dataloader(mean, std, batch_size, num_classes)
    cifar_test_loader = get_test_dataloader(mean, std, 100, num_classes)

    return cifar_train_loader, cifar_test_loader

In [ ]:
def train(epoch):
    start = time.time()
    net.train()
    train_loss = 0.0
    train_correct = 0.0
    for batch_index, (images, labels) in tqdm.tqdm(enumerate(cifar_train_loader)):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = outputs.max(1)
        train_correct += preds.eq(labels).sum()

        # print('Training Epoch: {epoch} [{trained_samples}/{total_samples}]\tLoss: {:0.4f}\tLR: {:0.6f}'.format(
        #     loss.item(),
        #     optimizer.param_groups[0]['lr'],
        #     epoch=epoch,
        #     trained_samples=batch_index * batch_size + len(images),
        #     total_samples=len(cifar_train_loader.dataset)
        # ))

    finish = time.time()

    train_error = 1 - train_correct.float().item() / len(cifar_train_loader.dataset)

    print()
    print('Training epoch: {}, LR: {:.6f}, Average loss: {:.4f}, Error: {:.4f}, Time consumed: {:.2f}s'.format(
        epoch,
        optimizer.param_groups[0]['lr'],
        train_loss / len(cifar_train_loader.dataset),
        train_error,
        finish - start
    ))

    return train_error

In [ ]:
def test(epoch):
    net.eval()

    test_correct_1 = 0.0
    test_correct_5 = 0.0

    with torch.no_grad():
        for batch_index, (images, labels) in enumerate(cifar_test_loader):
            images, labels = images.to(device), labels.to(device)

            outputs = net(images)
            _, preds = outputs.topk(5, 1, largest=True, sorted=True)

            labels = labels.view(labels.size(0), -1).expand_as(preds)
            correct = preds.eq(labels).float()

            test_correct_5 += correct[:, :5].sum()
            test_correct_1 += correct[:, :1].sum()

            test_top1_error = 1 - test_correct_1.item() / len(cifar_test_loader.dataset)
            test_top5_error = 1 - test_correct_5.item() / len(cifar_test_loader.dataset)

    print("Test")
    print("Top 1 err: {:.4f}".format(test_top1_error))
    print("Top 5 err: {:.4f}".format(test_top5_error))

    return test_top1_error, test_top5_error

In [ ]:
for order in range(3):
    print(order)

    num_classes, net = modelload()
    criterion, optimizer, scheduler, batch_size, num_epochs = utilload()
    cifar_train_loader, cifar_test_loader = dataload()

    RDMResNet56_Train_CIFAR10 = []
    RDMResNet56_TestTop1_CIFAR10 = []
    RDMResNet56_TestTop5_CIFAR10 = []

    for epoch in range(num_epochs):
        train_error = train(epoch)
        test_top1_error, test_top5_error = test(epoch)

        RDMResNet56_Train_CIFAR10.append(str(train_error))
        RDMResNet56_TestTop1_CIFAR10.append(str(test_top1_error))
        RDMResNet56_TestTop5_CIFAR10.append(str(test_top5_error))

        scheduler.step()

    with open(f"RDMResNet56 TrainingError CIFAR10 order{order}.txt", "w") as file1:
        file1.write('\n'.join(RDMResNet56_Train_CIFAR10))

    with open(f"RDMResNet56 TestError-Top1 CIFAR10 order{order}.txt", "w") as file2:
        file2.write('\n'.join(RDMResNet56_TestTop1_CIFAR10))

    with open(f"RDMResNet56 TestError-Top5 CIFAR10 order{order}.txt", "w") as file3:
        file3.write('\n'.join(RDMResNet56_TestTop5_CIFAR10))